In [20]:
import nltk
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import random
import numpy as np
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [21]:
def tokenize(sentance):
    return nltk.word_tokenize(sentance)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_word(tokenized_sentance,all_words):
    """
    sentance =["hello","how","are","you"]
    words = ["hi","hello","I","thank","cool","you","bye"]
    bow = [0,1,0,0,0,1,0]
    """

    tokenized_sentance = [stem(w) for w in tokenized_sentance]
    bag = np.zeros(len(all_words),dtype = np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_sentance:
            bag[idx]=1.0
    return bag
sentance =["hello","how","are","you"]
words = ["hi","hello","I","thank","cool","you","bye"]
bow = bag_of_word(sentance,words)
print(bow)
#words = ["organizes","organizer","organize"]
#stemming = [stem(w) for w in words]
#print(stemming)

[0. 1. 0. 0. 0. 1. 0.]


In [13]:
import json

with open('intents.json','r') as f:
    intents = json.load(f)
all_words = []
tags=[]
xy =[]

for intent in intents["intents"]:
    tag = intent["tag"]
    tags.append(tag)
    for pattern in intent["patterns"]:
        w=tokenize(pattern)
        all_words.extend(w)
        xy.append((w,tag))
ignore_words = ['?',',','!','.']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tag=sorted(set(tags))

x_train =[]
y_train=[]
for (pattern_sentance,tag) in xy:
    bag = bag_of_word(pattern_sentance,all_words)
    x_train.append(bag)

    label = tags.index(tag)
    y_train.append(label)
x_train = np.array(x_train)
y_train = np.array(y_train)

class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    def __len__(self):
        return self.n_samples

# Hyperparameters
batch_size = 8
hidden_size=8
input_size=len(x_train[0])
output_size=len(tags)
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,batch_size = batch_size,shuffle=True,num_workers=0)
epochs = 1000
learning_rate = 0.001

In [23]:
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size,hidden_size)
        self.l2 = nn.Linear(hidden_size,hidden_size)
        self.l3 = nn.Linear(hidden_size,num_classes)
        self.relu=nn.ReLU()

    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out
    
        

In [24]:
# training data
model = NeuralNet(input_size,hidden_size,output_size)
print(input_size,len(all_words))
print(output_size,tags)

49 49
6 ['greeting', 'goodbye', 'thanks', 'items', 'payments', 'Delivery']


In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size,hidden_size,output_size).to(device)
# calculating loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)


for epoch in range(epochs):
    for(words,labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Call the forward pass
        outputs = model(words)
        loss = criterion(outputs,labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch + 1)%100 == 0:
        print(f'epoch{epoch+1}/{epochs},loss = {loss.item():.4f}')
print(f'Final_Loss,loss={loss.item():.4f}')


epoch100/1000,loss = 1.3066
epoch200/1000,loss = 0.3280
epoch300/1000,loss = 0.0416
epoch400/1000,loss = 0.0248
epoch500/1000,loss = 0.0051
epoch600/1000,loss = 0.0028
epoch700/1000,loss = 0.0028
epoch800/1000,loss = 0.0018
epoch900/1000,loss = 0.0009
epoch1000/1000,loss = 0.0011
Final_Loss,loss=0.0011


In [26]:
# create dictionary

data = {
    "model_state" : model.state_dict(),
    "input_size" : input_size,
    "output_size":output_size,
    "hidden_size":hidden_size,
    "all_words":all_words,
    "tags":tags
}
FILE = "data.pth"
torch.save(data,FILE)

print(f'training complete. file saved to {FILE}')


In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
with open('intents.json','r') as f:
    intents = json.load(f)
FILE = "data.pth"
data = torch.load(FILE)
input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state=data["model_state"]
model = NeuralNet(input_size,hidden_size,output_size).to(device)
model.load_state_dict(model_state)

model.eval()

C:\Users\naguh\AppData\Local\Temp\ipykernel_17228\632071722.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(FILE)


NeuralNet(
  (l1): Linear(in_features=49, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=6, bias=True)
  (relu): ReLU()
)

In [2]:
bot = "sam"
def get_response(msg):
    sentance = tokenize(sentance)
    X = bag_of_word(sentance,all_words)
    X = X.reshape(1,X.shape[0])
    X=torch.from_numpy(X).to(device)

    output = model(X)
    _,predicted = torch.max(output,dim=1)
    tag = tags[predicted.item()]
    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                return random.choice(intent['responses'])
    else:
        return "I do not understand..."
    

    

In [49]:
# GUI Implementation
from tkinter import *
BG_GRAY = "#ABB2B9"
BG_COLOR = "#17202A"
TEXT_COLOR="#EAECEE"
FONT = ("Times New Roman",12)
FONT_BOLD =("Times New Roman ",12,"bold")

class application:

    def __init__(self):
        self.window = Tk()
        self._setup_main_window()
    def run(self):
        self.window.mainloop()
    def _setup_main_window(self):
        self.window.title("Chat")
        self.window.resizable(width=False,height = False)
        self.window.configure(width = 470,height = 550,bg=BG_COLOR)
        # naming head Label
        head_label = Label(self.window,bg = BG_COLOR,fg = TEXT_COLOR,text = "Welcome to chatbot",font = FONT_BOLD,pady = 10)
        head_label.place(relwidth=1)

        line = Label(self.window,width = 450,bg = BG_GRAY)
        line.place(relwidth=1,rely=0.07,relheight=0.012)

        self.text_widget = Text(self.window,width = 20,height = 2,bg = BG_COLOR,fg = TEXT_COLOR,font = FONT,padx = 5,pady = 5)
        self.text_widget.place(relheight = 0.745,relwidth = 1,rely = 0.08)
        self.text_widget.configure(cursor="arrow",state=DISABLED)
        
        
if __name__ == "__main__":
    app = application()
    app.run()
        
        